In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
import netCDF4
import cartopy
from pyproj import CRS, Proj, transform, Transformer
import numpy as np
import mapclassify
import pysal
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

/apps/geopython/1.0.1/lib/python3.7/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


### below creates a df categories, damages, and counts of damaging events


In [2]:
StormData=gpd.read_file(r'/blue/emullens/meirahwilliamson/StormData_shapefile_2/StormData.shp')

In [3]:
# fill nas with 0 (nas exist if a county never has ff damage)
StormData['date']=StormData['date'].fillna(0)

In [4]:
# create season function
def choose_season(season):
    csv=pd.read_csv(r'/blue/emullens/meirahwilliamson/netcdf_ero/'+str(season)+'/'+str(season)+'.csv')
    csv_fips_index=csv.set_index('FIPS')
    return csv_fips_index
    

In [5]:
#add category to each place
category_list=[]
# check each row, find date, geoid, and season
for i in StormData.iterrows():
    row=i[1]
    geoid=int(row['GEOID'])
    date=int(row['date'])
    season=row['season']
    # no season (i.e. no damages, append with -99)
    if season == '' or season==None:
        category_list.append(-99)
    else:
        csv=choose_season(season)
        # mising netcdf
        if str(date) not in csv:
            category_list.append(-50)
        #netcdf is there and there are damages yay
        else:    
            category_list.append(csv[str(date)][geoid])

In [6]:
#add new category column
StormData['category']=category_list

In [7]:
# make sure it worked
x=StormData['category'].unique()
list(x)

[0.5,
 -99.0,
 0.05000000074505806,
 0.0,
 0.10000000149011612,
 0.20000000298023224,
 -50.0,
 nan]

In [8]:
StormData_nan=StormData.dropna()
StormData_v2=StormData_nan[~(StormData_nan['episode_na'].str.contains("Harvey|harvey"))]

In [9]:
StormData_v2.to_file(r'/blue/emullens/meirahwilliamson/StormData_shapefile_noharvey_riskcat/StormData_nh_rc.shp')

In [13]:
StormData_v2=gpd.read_file(r'/blue/emullens/meirahwilliamson/StormData_shapefile_noharvey_riskcat/StormData_nh_rc.shp')

In [15]:
# make sure it worked
x=StormData_v2['category'].unique()
list(x)

[0.5, 0.050000000745058, 0.0, 0.100000001490116, 0.200000002980232, -50.0]

In [14]:
plt.rcParams.keys()
params = {'axes.labelsize': 30,
          'axes.titlesize': 34,
          'axes.titlepad': 2,
          'xtick.labelsize': 26,
         'ytick.labelsize': 26,
         'legend.title_fontsize':28,
         'legend.fontsize':20}
plt.rcParams.update(params)

subplot_labels=['a)','b)','c)','d)']

### plot causes


In [60]:
fig.savefig(r'/blue/emullens/meirahwilliamson/figures/damage_attribution_risks.png',dpi=300)


### below creates gdfs for each category

In [22]:
counties=gpd.read_file(r'/blue/emullens/meirahwilliamson/netcdf_ero/county_data/USA_Counties/USA_Counties.shp')

In [23]:
# counties=counties[~(counties['STATE_NAME']=='Puerto Rico') & ~(counties['STATE_NAME']=='Alaska')
#         & ~(counties['STATE_NAME']=='Hawaii')]
# counties=counties.drop(counties.columns[0:6],axis=1)
# counties=counties.drop(counties.columns[1:50],axis=1)
counties=counties.to_crs(3857)

In [35]:
counties=gpd.read_file(r'/blue/emullens/meirahwilliamson/Shapefiles/US_Counties/US_Counties.shp')
counties=counties_2.to_crs(3857)

In [3]:
risk_levels_dict={'marginal':0.05000000074505806,
                      'slight':0.10000000149011612,
                      'moderate':0.20000000298023224,
                      'high':0.5}

In [42]:
#create gdfs for each category
for i in risk_levels_dict:
    category=StormData_group_sum_df[StormData_group_sum_df['category']==risk_levels_dict[i]]
    merged=pd.merge(counties,category, how='left',left_on="GEOID",right_on='GEOID')
    merged.to_file(r'/blue/emullens/meirahwilliamson/StormData_Risk_Categories/'+i+'_damaged_2/'+i+'.shp')
    

### get some stats

In [6]:
for risk_level in risk_levels_dict:
    damaged_counties=gpd.read_file(r'/blue/emullens/meirahwilliamson/StormData_Risk_Categories/'+risk_level+'_damaged/'+risk_level+'.shp')
    
    mean=damaged_counties['SUM_proper'].mean()
    mean_2=damaged_counties[damaged_counties['SUM_proper']!=0]['SUM_proper'].mean()
    print(damaged_counties['SUM_proper'].sum())

    median=damaged_counties['SUM_proper'].median()
    median_2=damaged_counties[damaged_counties['SUM_proper']!=0]['SUM_proper'].median()

    
    mode=damaged_counties['SUM_proper'].mode()
    mode_2=damaged_counties[damaged_counties['SUM_proper']!=0]['SUM_proper'].mode()

    print(risk_level,mean_2,median_2)

285571020.0
marginal 477543.51170568564 25000.0
858195680.0
slight 1178840.2197802197 45000.0
2953485130.0
moderate 8136322.672176309 50000.0
42114024860.0
high 244848981.74418604 125000.0


In [7]:
damaged_counties

,FIPS,SHAPE_Le_1,SHAPE_Area,GEOID,category,SUM_proper,SUM_crop_d,SUM_prop_c,count,geometry
0,06053,6.495277,0.860396,None,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-13520087.681 4283419.790, -13..."
1,06087,2.388135,0.117218,None,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13605624.004 4469113.195, -13605618..."
2,06085,3.933257,0.341450,None,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-13539883.406 4506615.125, -13..."
3,06069,3.718732,0.362726,None,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13534222.246 4424930.701, -13534210..."
4,06111,3.916199,0.471859,None,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-13306531.467 3933240.934, -13..."
...,...,...,...,...,...,...,...,...,...,...
3103,23025,8.024429,1.221142,None,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-7794866.037 5873433.117, -7794461.8..."
3104,23003,8.081154,2.078769,None,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-7703796.789 6016230.645, -7703740.6..."
3105,23019,6.546183,1.058448,None,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-7660933.441 5843985.707, -7657826.4..."
3106,23021,6.004977,1.313017,None,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-7761311.791 5872791.981, -7751600.8..."


In [15]:
x=[]
for risk_level in risk_levels_dict:
    damaged_counties=gpd.read_file(r'/blue/emullens/meirahwilliamson/StormData_Risk_Categories/'+risk_level+'_damaged_2/'+risk_level+'.shp')
    
    mean=damaged_counties['property_d'].mean()
    mean_2=damaged_counties[damaged_counties['property_d']!=0]['property_d'].mean()

    median=damaged_counties['property_d'].median()
    median_2=damaged_counties[damaged_counties['property_d']!=0]['property_d'].median()

    
    mode=damaged_counties['property_d'].mode()
    mode_2=damaged_counties[damaged_counties['property_d']!=0]['property_d'].mode()

    x.append(damaged_counties[damaged_counties['property_d']>0]['property_d'])
    
    
    
    print(risk_level,mean_2,median_2)#,fvalue,pvalue)

marginal 477543.51170568564 25000.0
slight 1178840.2197802197 45000.0
moderate 8136322.672176309 50000.0
high 244848981.74418604 125000.0


In [5]:
x=[]
for risk_level in risk_levels_dict:
    damaged_counties=gpd.read_file(r'/blue/emullens/meirahwilliamson/StormData_Risk_Categories/'+risk_level+'_damaged_2/'+risk_level+'.shp')
    
    mean=damaged_counties['property_d'].mean()

    median=damaged_counties['property_d'].median()
    
    mode=damaged_counties['property_d'].mode()

    #x.append(damaged_counties[damaged_counties['property_d']>0]['property_d'])
    
    
    
    print(risk_level,mean,median)

marginal 477543.51170568564 25000.0
slight 1178840.2197802197 45000.0
moderate 8136322.672176309 50000.0
high 244848981.74418604 125000.0


### missed damaged counties

In [130]:
StormData=gpd.read_file(r'/blue/emullens/meirahwilliamson/StormData_shapefile_2/StormData.shp')

In [131]:
StormData['date']=StormData['date'].fillna(0)

In [132]:
# create season function
def choose_season(season):
    csv=pd.read_csv(r'/blue/emullens/meirahwilliamson/netcdf_ero/'+str(season)+'/'+str(season)+'.csv')
    csv_fips_index=csv.set_index('FIPS')
    return csv_fips_index
    

In [69]:
category_list=[]
# check each row, find date, geoid, and season
for i in StormData.iterrows():
    row=i[1]
    geoid=int(row['GEOID'])
    date=int(row['date'])
    season=row['season']
    # no season (i.e. no damages, append with -99)
    if season == '' or season==None:
        category_list.append(-99)
    else:
        csv=choose_season(season)
        # mising netcdf
        if str(date) not in csv:
            category_list.append(-50)
        #netcdf is there and there are damages yay
        else:    
            category_list.append(csv[str(date)][geoid])

In [19]:
#add new category column
StormData['category']=category_list

In [20]:
# get sums of damages and count how many times damages occurred 
StormData_group_sum_df=StormData.groupby(['GEOID','category'])['property_d'].sum().reset_index()
StormData_group_count_df=StormData.groupby(['GEOID','category'])['property_d'].count().reset_index()

In [21]:
StormData_group_sum_df['count']=StormData_group_count_df['property_d']